In [9]:
import os, requests
from google.oauth2 import service_account
from google.auth.transport.requests import AuthorizedSession

In [10]:
import os, requests
from google.oauth2 import service_account
from google.auth.transport.requests import AuthorizedSession
creds_dir = "/Users/yukachen/marketing-operation/credential_api"
print(os.listdir(creds_dir))
import keyring

token_file = "/Users/yukachen/marketing-operation/credential_api/token.json"

# Positional args: (service_name, username, password)
keyring.set_password(
    "googlesheets4",
    "token_path",
    token_file
)
print(keyring.get_password("googlesheets4", "token_path"))
import keyring

# These exactly mirror what you set in R:
client_secret_path = keyring.get_password("googlesheets4", "client_secret_path")
token_path         = keyring.get_password("googlesheets4", "token_path")
email              = keyring.get_password("googlesheets4", "email")

print(client_secret_path)  # -> "/Users/yukachen/.../client_secret.json"
print(token_path)          # -> "/Users/yukachen/.../token.json"
print(email)               # -> "youjia.chen@wondersco.com"

from google.oauth2 import service_account
from google.auth.transport.requests import AuthorizedSession

SA_KEY = keyring.get_password("gcp_service_account", "mactarro_key_path")
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

creds   = service_account.Credentials.from_service_account_file(SA_KEY, scopes=SCOPES)
session = AuthorizedSession(creds)
# → fully headless access under your mactarro service account

['mkt-weekly-data-bfca4760de89.json', 'token.json', 'client_secret_1063101091245-kj6kdb5d4gefva4khlnp3juarhsqn0h5.apps.googleusercontent.com.json', 'user_token_gsheets.json', 'user_token.json']
/Users/yukachen/marketing-operation/credential_api/token.json
/Users/yukachen/marketing-operation/credential_api/client_secret_1063101091245-kj6kdb5d4gefva4khlnp3juarhsqn0h5.apps.googleusercontent.com.json
/Users/yukachen/marketing-operation/credential_api/token.json
youjia.chen@wondersco.com


### 2025

In [11]:
import os
import re
import csv
import keyring
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

# ————————————————
# 0) map each month to its Spreadsheet ID
# ————————————————
MONTH_SPREADSHEETS = {
    "2025-07":"1zQZBcNV_rJtyjXc3hr5mYJHBqytMQb4gqW9Do3ulBJk",
    "2025-06":"1JANcc8X-2oF_P16ZgbltH8u5fuSUh6bf5KQI60IVqek",
    "2025-05":"1ahg70Ug5IMRNTj8EsUXeaHnhDuM2WSN1BuIFI0buT1w",
    "2025-04":"1ZsZeE2aWaY64mza1nPagbFz2DWJUaZv-e0aC8OkHPeM",
    "2025-03":"137jja3Xq7j1WI5HPBzIjYGr3lOSfDuDrNN7Fjn7J_9Q",
    "2025-02":"1RQqRGXZ4pFTYGJ-v1nh52RltlbZJ9VJXDSv3shjVmuA",
    "2025-01":"1yp8gzvmjZpDIpfzGBuC3o5oh38lr3KAWJ_aW-Lnxnrs",

}

# ————————————————
# 1) OAuth under your user (one-time consent)
# ————————————————
CLIENT_SECRET = keyring.get_password("googlesheets4", "client_secret_path")
TOKEN_PATH    = os.path.expanduser(
    "~/marketing-operation/credential_api/user_token.json"
)
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

def get_user_creds():
    creds = None
    if os.path.exists(TOKEN_PATH):
        creds = Credentials.from_authorized_user_file(TOKEN_PATH, SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET, SCOPES)
            creds = flow.run_local_server(port=0)  # one-time browser pop-up
        os.makedirs(os.path.dirname(TOKEN_PATH), exist_ok=True)
        with open(TOKEN_PATH, "w") as f:
            f.write(creds.to_json())
    return creds

creds   = get_user_creds()
service = build("sheets", "v4", credentials=creds)

# ————————————————
# 2) Exclusion rules
# ————————————————
EXACT_EXCLUDE = {
    "summary", "existing client", "churn 1y",
    "active leads", "active opp", "active account contact",
    "unqualified leads", "opt-out"
}

SUBSTR_EXCLUDE = {
    "exclusion",
    "language",
    "snowflake",
    "50char",
    "lead score",
    "enrichment",
    "thai",         # catches Thai_Viet, Thai-Viet, Thai Viet, etc.
    "existing",
    "sheet",
    "scoring",
    "all",
    "opt",
    "denver",
    "chicago",
    "city"
}

def normalize_title(title: str) -> str:
    # 1) strip any .csv suffix
    t = re.sub(r"(?i)\.csv$", "", title)
    # 2) lowercase
    t = t.lower()
    # 3) replace underscores/hyphens with spaces
    t = t.replace("_", " ").replace("-", " ")
    # 4) collapse multiple spaces to single
    t = re.sub(r"\s+", " ", t).strip()
    return t

# ————————————————
# 3) Prepare output
# ————————————————
os.makedirs("list_exports", exist_ok=True)

# ————————————————
# 4) Batch export
# ————————————————
for month, ssid in MONTH_SPREADSHEETS.items():
    meta = service.spreadsheets().get(
        spreadsheetId=ssid,
        fields="sheets(properties(sheetId,title))"
    ).execute()

    for sheet in meta["sheets"]:
        title = sheet["properties"]["title"]
        norm  = normalize_title(title)

        # skip exact or substring matches
        if norm in EXACT_EXCLUDE or any(sub in norm for sub in SUBSTR_EXCLUDE):
            continue

        safe_title = title.replace("/", "_")
        rows = service.spreadsheets().values().get(
            spreadsheetId=ssid,
            range=f"'{title}'"
        ).execute().get("values", [])

        out_path = os.path.join("list_exports", f"{month}—{safe_title}.csv")
        with open(out_path, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerows(rows)

        print(f"WROTE {out_path} ({len(rows)} rows)")

WROTE list_exports/2025-07—A-LaborHighGMV.csv (5001 rows)
WROTE list_exports/2025-07—B-RevenueHighGMV.csv (5001 rows)
WROTE list_exports/2025-07—C-LaborLowGMV.csv (14863 rows)
WROTE list_exports/2025-07—D-RevenueLowGMV.csv (14863 rows)
WROTE list_exports/2025-06—Group A - Chinese.csv (20471 rows)
WROTE list_exports/2025-06—Group B - Japanese.csv (17074 rows)
WROTE list_exports/2025-06—Brizo Full List .csv (37544 rows)
WROTE list_exports/2025-05—Group A - High GMV.csv (5101 rows)
WROTE list_exports/2025-05—Group B - Low GMV.csv (45256 rows)
WROTE list_exports/2025-04—Group A - Giftcard (1-25601).csv (25602 rows)
WROTE list_exports/2025-04—Group B - Air Purifier (25602-51230).csv (25630 rows)
WROTE list_exports/2025-03—A - Credibility (1-22406).csv (22407 rows)
WROTE list_exports/2025-03—B - Heartfelt (22407-44812).csv (22407 rows)
WROTE list_exports/2025-02—Drop1A - Chinese VO.csv (6231 rows)
WROTE list_exports/2025-02—Drop1B - Chinese DE.csv (6279 rows)
WROTE list_exports/2025-02—Drop1

## 2024

In [12]:
import os
import re
import csv
import keyring
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

# ————————————————
# 0) map each month to its Spreadsheet ID
# ————————————————
MONTH_SPREADSHEETS = {
    "2024-12":"1BQiShHjD-yzlKT7gRseVTBCNRxtpPhZmUs4STgfXmlA",
    "2024-11":"152E4ZlzmzZSKmhZcjJshrV4UXH2r6bqbjqzk3FQNcl0",
    "2024-10":"12Qbdjdw5lUg6dJQbdSxZRQ-pYnK1d8AdDHNJ1nt90i0",
    "2024-09":"19MW0LJCKDLvgpX8Mx505QloIM67iwW7z3yfuW9fEu90",
    "2024-08":"1RxFZIzXITCy6z3H-hfk_DMz3QCLU-MakjKLJf9YPrYA",
    "2024-07":"1XU-Ndmtf54w5IAg6E_7eeGCJ_t2e-Q-J16DK8LafUPg",
    "2024-06":"1uIWdtPEVT1FIk4AzvRJMGfKtyQHU_QuAXCQFDbvfVRI",
    "2024-05":"1tbNmGqOl-TIp3k_LXVc2hnM1oCmwvtZYGQHy-_ilasA",
    "2024-04":"1nn43iw02426KVL8jSICB3qdfzwMEY-sylsnmMki_Pgw",
    "2024-03":"1To8datVyTFD5jZgVANgXMjSkk3uO1d_KH1cfxNRyJMU",
    "2024-02":"1xMpiBGoLdWZz6r4J22RfAH8t22dpZ_PY_zN6b5dxgmQ",
    "2024-01":"1hww5FsbAVqjyGjnrs7sI2hB-Em6fmFgvlWfpsuvIxmI",

}

# ————————————————
# 1) OAuth under your user (one-time consent)
# ————————————————
CLIENT_SECRET = keyring.get_password("googlesheets4", "client_secret_path")
TOKEN_PATH    = os.path.expanduser(
    "~/marketing-operation/credential_api/user_token.json"
)
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

def get_user_creds():
    creds = None
    if os.path.exists(TOKEN_PATH):
        creds = Credentials.from_authorized_user_file(TOKEN_PATH, SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET, SCOPES)
            creds = flow.run_local_server(port=0)  # one-time browser pop-up
        os.makedirs(os.path.dirname(TOKEN_PATH), exist_ok=True)
        with open(TOKEN_PATH, "w") as f:
            f.write(creds.to_json())
    return creds

creds   = get_user_creds()
service = build("sheets", "v4", credentials=creds)

# ————————————————
# 2) Exclusion rules
# ————————————————
EXACT_EXCLUDE = {
    "summary", "existing client", "churn 1y",
    "active leads", "active opp", "active account contact",
    "unqualified leads", "opt-out"
}

SUBSTR_EXCLUDE = {
    "exclusion",
    "language",
    "snowflake",
    "50char",
    "lead score",
    "enrichment",
    "thai",         # catches Thai_Viet, Thai-Viet, Thai Viet, etc.
    "existing",
    "sheet",
    "scoring",
    "all",
    "opt",
    "denver",
    "chicago",
    "city"
}

def normalize_title(title: str) -> str:
    # 1) strip any .csv suffix
    t = re.sub(r"(?i)\.csv$", "", title)
    # 2) lowercase
    t = t.lower()
    # 3) replace underscores/hyphens with spaces
    t = t.replace("_", " ").replace("-", " ")
    # 4) collapse multiple spaces to single
    t = re.sub(r"\s+", " ", t).strip()
    return t

# ————————————————
# 3) Prepare output
# ————————————————
os.makedirs("list_exports", exist_ok=True)

# ————————————————
# 4) Batch export
# ————————————————
for month, ssid in MONTH_SPREADSHEETS.items():
    meta = service.spreadsheets().get(
        spreadsheetId=ssid,
        fields="sheets(properties(sheetId,title))"
    ).execute()

    for sheet in meta["sheets"]:
        title = sheet["properties"]["title"]
        norm  = normalize_title(title)

        # skip exact or substring matches
        if norm in EXACT_EXCLUDE or any(sub in norm for sub in SUBSTR_EXCLUDE):
            continue

        safe_title = title.replace("/", "_")
        rows = service.spreadsheets().values().get(
            spreadsheetId=ssid,
            range=f"'{title}'"
        ).execute().get("values", [])

        out_path = os.path.join("list_exports", f"{month}—{safe_title}.csv")
        with open(out_path, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerows(rows)

        print(f"WROTE {out_path} ({len(rows)} rows)")

WROTE list_exports/2024-12—A - Chinese _Unknown.csv (41285 rows)
WROTE list_exports/2024-12—B - English.csv (8784 rows)
WROTE list_exports/2024-11—Chinese (1-20118).csv (19841 rows)
WROTE list_exports/2024-11—English (20119-28986).csv (8814 rows)
WROTE list_exports/2024-11—Unknown (28987-50734).csv (21749 rows)
WROTE list_exports/2024-11—2nd DM - Chinese.csv (9821 rows)
WROTE list_exports/2024-11—2nd DM - English_Other.csv (11334 rows)
WROTE list_exports/2024-11—20k States.csv (16 rows)
WROTE list_exports/2024-10—A - Western 1-26130.csv (26131 rows)
WROTE list_exports/2024-10—B - Chinese 26131-51680.csv (25551 rows)
WROTE list_exports/2024-09—Brizo A (Creative) 1-24229.csv (24230 rows)
WROTE list_exports/2024-09—Brizo B (Evergreen) 24230-49312.csv (25084 rows)
WROTE list_exports/2024-08—Brizo - A ($100) 1-18487.csv (18488 rows)
WROTE list_exports/2024-08—Brizo - B ($200) 18488-35526.csv (17040 rows)
WROTE list_exports/2024-08—Brizo - C ($300) 35527-51230.csv (15705 rows)
WROTE list_exp